In [40]:
#Import libraries 
import arcpy 
import pandas as pd
import numpy as np

In [41]:
#Connect to workspace
arcpy.env.workspace = "G:\SSCI575_Projects\Project2_dataset.gdb\Project2_dataset.gdb"

In [42]:
#List all the features classes in the geodatabase
arcpy.ListFeatureClasses()

['kc_house_data']

In [43]:
#list all the fields in the dataset
for i in range(len(arcpy.ListFields('kc_house_data'))):
    print(arcpy.ListFields('kc_house_data')[i].name)

OBJECTID
Shape
id
date
price
bedrooms
bathrooms
sqft_living
sqft_lot
floors
waterfront
view
condition
grade
sqft_above
sqft_basement
yr_built
yr_renovated
zipcode
lat
long
sqft_living15
sqft_lot15


In [44]:
#Read shapefile and add all values to a dataframe
shapefile_path = 'kc_house_data'
fields = [f.name for f in arcpy.ListFields(shapefile_path)]
rows = []

with arcpy.da.SearchCursor(shapefile_path, fields) as cursor:
    for row in cursor:
        rows.append(row)

In [45]:
#Save the dataframe as df
df = pd.DataFrame(rows, columns=fields)
df

,OBJECTID,Shape,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1,"(-122.25699999999995, 47.51120000000003)",7.129301e+09,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,2,"(-122.31899999999996, 47.72100000000006)",6.414100e+09,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,3,"(-122.23299999999995, 47.73790000000008)",5.631500e+09,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,4,"(-122.39299999999997, 47.520800000000065)",2.487201e+09,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,5,"(-122.04499999999996, 47.61680000000007)",1.954401e+09,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,21609,"(-122.34599999999995, 47.69930000000005)",2.630000e+08,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,21610,"(-122.36199999999997, 47.51070000000004)",6.600060e+09,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,21611,"(-122.29899999999998, 47.594400000000064)",1.523300e+09,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,21612,"(-122.06899999999996, 47.53450000000004)",2.913101e+08,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [46]:
#Get correlation matrix to investigate which attributes are highly correlated to house price
matrix = df.corr()
matrix

,OBJECTID,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
OBJECTID,1.000000,0.006718,0.027372,0.010794,0.107334,0.044717,-0.026877,0.179138,-0.007611,-0.013751,-0.095419,0.082050,0.071958,-0.041841,0.199549,-0.025601,-0.002199,0.000701,0.008649,0.026647,-0.029827
id,0.006718,1.000000,-0.016762,0.001286,0.005160,-0.012258,-0.132109,0.018525,-0.002721,0.011592,-0.023783,0.008130,-0.010842,-0.005151,0.021380,-0.016907,-0.008224,-0.001891,0.020799,-0.002901,-0.138798
price,0.027372,-0.016762,1.000000,0.308350,0.525138,0.702035,0.089661,0.256794,0.266369,0.397293,0.036362,0.667434,0.605567,0.323816,0.054012,0.126434,-0.053203,0.307003,0.021626,0.585379,0.082447
bedrooms,0.010794,0.001286,0.308350,1.000000,0.515884,0.576671,0.031703,0.175429,-0.006582,0.079532,0.028472,0.356967,0.477600,0.303093,0.154178,0.018841,-0.152668,-0.008931,0.129473,0.391638,0.029244
bathrooms,0.107334,0.005160,0.525138,0.515884,1.000000,0.754665,0.087740,0.500653,0.063744,0.187737,-0.124982,0.664983,0.685342,0.283770,0.506019,0.050739,-0.203866,0.024573,0.223042,0.568634,0.087175
sqft_living,0.044717,-0.012258,0.702035,0.576671,0.754665,1.000000,0.172826,0.353949,0.103818,0.284611,-0.058753,0.762704,0.876597,0.435043,0.318049,0.055363,-0.199430,0.052529,0.240223,0.756420,0.183286
sqft_lot,-0.026877,-0.132109,0.089661,0.031703,0.087740,0.172826,1.000000,-0.005201,0.021604,0.074710,-0.008958,0.113621,0.183512,0.015286,0.053080,0.007644,-0.129574,-0.085683,0.229521,0.144608,0.718557
floors,0.179138,0.018525,0.256794,0.175429,0.500653,0.353949,-0.005201,1.000000,0.023698,0.029444,-0.263768,0.458183,0.523885,-0.245705,0.489319,0.006338,-0.059121,0.049614,0.125419,0.279885,-0.011269
waterfront,-0.007611,-0.002721,0.266369,-0.006582,0.063744,0.103818,0.021604,0.023698,1.000000,0.401857,0.016653,0.082775,0.072075,0.080588,-0.026161,0.092885,0.030285,-0.014274,-0.041910,0.086463,0.030703
view,-0.013751,0.011592,0.397293,0.079532,0.187737,0.284611,0.074710,0.029444,0.401857,1.000000,0.045990,0.251321,0.167649,0.276947,-0.053440,0.103917,0.084827,0.006157,-0.078400,0.280439,0.072575


In [47]:
#Evaluate all the attributes highly correlated to price
corr_price = matrix['price']
corr_price

OBJECTID         0.027372
id              -0.016762
price            1.000000
bedrooms         0.308350
bathrooms        0.525138
sqft_living      0.702035
sqft_lot         0.089661
floors           0.256794
waterfront       0.266369
view             0.397293
condition        0.036362
grade            0.667434
sqft_above       0.605567
sqft_basement    0.323816
yr_built         0.054012
yr_renovated     0.126434
zipcode         -0.053203
lat              0.307003
long             0.021626
sqft_living15    0.585379
sqft_lot15       0.082447
Name: price, dtype: float64

In [48]:
#Filter for highly correlated values
highly_corr_attributes = corr_price[abs(corr_price) > 0.5].index

In [49]:
#Create a new dataframe from highly correlated attributes
df_highly_corr = df[highly_corr_attributes]

In [50]:
#Find r-squared values from highly correlated attributes
r_squared = df_highly_corr.corr() ** 2
r_squared

,price,bathrooms,sqft_living,grade,sqft_above,sqft_living15
price,1.000000,0.275769,0.492853,0.445468,0.366712,0.342668
bathrooms,0.275769,1.000000,0.569520,0.442202,0.469694,0.323345
sqft_living,0.492853,0.569520,1.000000,0.581718,0.768422,0.572172
grade,0.445468,0.442202,0.581718,1.000000,0.571419,0.508657
sqft_above,0.366712,0.469694,0.768422,0.571419,1.000000,0.535634
sqft_living15,0.342668,0.323345,0.572172,0.508657,0.535634,1.000000


In [51]:
multicol_vals = r_squared[abs(r_squared) > 0.5].index
multicol_vals

Index(['price', 'bathrooms', 'sqft_living', 'grade', 'sqft_above',
       'sqft_living15'],
      dtype='object')

In [52]:
# Set your threshold
threshold = 0.75

# Find and print collinear pairs
for i in range(len(r_squared.columns)):
    for j in range(i+1, len(r_squared.columns)):
        if r_squared.iloc[i, j] > threshold:
            print(f"{r_squared.columns[i]} and {r_squared.columns[j]}: R² = {r_squared.iloc[i, j]:.2f}")

sqft_living and sqft_above: R² = 0.77


In [53]:
#Read shapefile and add all values to a dataframe for high house price clusters
shapefile_path = 'kc_house_data_highprice'
fields = [f.name for f in arcpy.ListFields(shapefile_path)]
rows = []

with arcpy.da.SearchCursor(shapefile_path, fields) as cursor:
    for row in cursor:
        rows.append(row)

In [54]:
#Save the dataframe as df for high house price clusters
highprice_df = pd.DataFrame(rows, columns=fields)
highprice_df

,OBJECTID,Shape,FID_kc_house_data,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,FID_RasterT_Extract1,Id_1,gridcode
0,1,"(-122.34799999999996, 47.416900000000055)",541,6.220491e+08,20150218T000000,2125000.0,3,2.50,5403,24069,2.0,1,4,4,12,5403,0,1976,0,98166,47.4169,-122.348,3980,104374,58,58,1
1,2,"(-121.90499999999997, 47.50160000000005)",20472,3.230791e+08,20150123T000000,1800000.0,4,3.50,6370,205603,2.0,0,0,3,12,6370,0,2008,0,98027,47.5016,-121.905,1490,33580,57,57,1
2,3,"(-122.20499999999998, 47.52830000000006)",20326,5.185005e+08,20140811T000000,3000000.0,3,3.50,4410,10756,2.0,1,4,3,11,3430,980,2014,0,98056,47.5283,-122.205,3550,5634,54,54,1
3,4,"(-122.20499999999998, 47.528500000000065)",21202,5.185005e+08,20141008T000000,2230000.0,3,3.50,3760,5634,2.0,1,4,3,11,2830,930,2014,0,98056,47.5285,-122.205,3560,5762,54,54,1
4,5,"(-122.21999999999997, 47.52870000000007)",9176,3.124059e+09,20150213T000000,1955000.0,3,1.75,3330,12566,1.0,1,4,4,8,1940,1390,1960,0,98040,47.5287,-122.220,3730,16560,53,53,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,202,"(-122.37199999999996, 47.72950000000003)",14557,2.303900e+09,20140611T000000,2888000.0,5,6.25,8670,64033,2.0,0,4,3,13,6120,2550,1965,2003,98177,47.7295,-122.372,4140,81021,5,5,1
202,203,"(-122.36999999999995, 47.72960000000006)",19018,2.303900e+09,20140911T000000,3800000.0,3,4.25,5510,35000,2.0,0,4,3,13,4910,600,1997,0,98177,47.7296,-122.370,3430,45302,5,5,1
203,204,"(-122.36999999999995, 47.73130000000003)",2126,4.379600e+09,20140729T000000,1325000.0,3,3.75,6400,76665,1.0,0,2,4,10,3810,2590,1966,0,98177,47.7313,-122.370,3430,60548,5,5,1
204,205,"(-122.36199999999997, 47.733400000000074)",13412,2.426039e+09,20150130T000000,2415000.0,5,4.75,7880,24250,2.0,0,2,3,13,7880,0,1996,0,98177,47.7334,-122.362,2740,10761,4,4,1


In [55]:
#Get correlation matrix to investigate which attributes are highly correlated to house price
matrix = highprice_df.corr()
matrix

,OBJECTID,FID_kc_house_data,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,FID_RasterT_Extract1,Id_1,gridcode
OBJECTID,1.000000,-0.147540,-0.011098,0.016389,0.032665,-0.058985,-0.016779,-0.103410,0.020972,-0.082023,-0.132940,0.032868,-0.021670,-0.015854,-0.007926,-0.173863,0.069094,0.294086,0.913537,-0.367314,-0.129066,-0.058385,-0.689768,-0.689768,NaN
FID_kc_house_data,-0.147540,1.000000,-0.146053,-0.131195,0.068783,0.076122,-0.048718,-0.005324,0.074126,-0.094300,-0.038316,-0.119460,0.059506,0.005984,-0.110104,0.229304,-0.006751,-0.138201,-0.106364,0.135399,-0.008541,-0.018592,0.073158,0.073158,NaN
id,-0.011098,-0.146053,1.000000,0.112857,0.049325,-0.012987,0.001298,-0.091607,0.062839,-0.048831,-0.000148,-0.026720,-0.093548,0.024075,-0.037644,-0.077348,0.062552,-0.213073,0.022015,-0.048711,-0.082366,-0.099630,0.021832,0.021832,NaN
price,0.016389,-0.131195,0.112857,1.000000,0.384827,0.663274,0.745850,0.014433,0.232569,0.340361,0.311506,-0.118556,0.616815,0.676547,0.399489,0.215657,0.057661,-0.041082,0.036273,0.054116,0.246058,0.027030,-0.010785,-0.010785,NaN
bedrooms,0.032665,0.068783,0.049325,0.384827,1.000000,0.515497,0.502059,-0.045262,0.142002,-0.058628,0.046716,0.019115,0.281738,0.437090,0.299582,0.033805,0.053059,-0.147247,0.027740,0.048563,0.186804,-0.010106,-0.048218,-0.048218,NaN
bathrooms,-0.058985,0.076122,-0.012987,0.663274,0.515497,1.000000,0.802171,0.091093,0.288094,0.098004,0.219494,-0.208853,0.644044,0.720361,0.441834,0.320657,0.000535,0.038324,-0.026284,0.106887,0.190320,0.114642,-0.011988,-0.011988,NaN
sqft_living,-0.016779,-0.048718,0.001298,0.745850,0.502059,0.802171,1.000000,0.181192,0.319987,0.135304,0.241816,-0.155354,0.730959,0.875633,0.588273,0.279829,0.045099,0.006575,-0.014070,0.188256,0.272369,0.233986,-0.039693,-0.039693,NaN
sqft_lot,-0.103410,-0.005324,-0.091607,0.014433,-0.045262,0.091093,0.181192,1.000000,0.089592,-0.043904,-0.031356,-0.091314,0.200426,0.214482,0.013123,0.146797,-0.031448,0.002513,-0.122349,0.527649,-0.038311,0.935287,0.091347,0.091347,NaN
floors,0.020972,0.074126,0.062839,0.232569,0.142002,0.288094,0.319987,0.089592,1.000000,0.095722,0.122724,-0.096966,0.396114,0.473091,-0.134737,0.123520,0.026325,0.203403,-0.000446,-0.056072,-0.095291,0.085629,-0.008468,-0.008468,NaN
waterfront,-0.082023,-0.094300,-0.048831,0.340361,-0.058628,0.098004,0.135304,-0.043904,0.095722,1.000000,0.589191,-0.053133,0.126461,0.062090,0.174007,0.218055,-0.016217,-0.009344,-0.082452,0.197282,-0.007206,-0.003821,-0.009282,-0.009282,NaN


In [56]:
#Evaluate all the attributes highly correlated to price
corr_price_highprice = matrix['price']
corr_price_highprice

OBJECTID                0.016389
FID_kc_house_data      -0.131195
id                      0.112857
price                   1.000000
bedrooms                0.384827
bathrooms               0.663274
sqft_living             0.745850
sqft_lot                0.014433
floors                  0.232569
waterfront              0.340361
view                    0.311506
condition              -0.118556
grade                   0.616815
sqft_above              0.676547
sqft_basement           0.399489
yr_built                0.215657
yr_renovated            0.057661
zipcode                -0.041082
lat                     0.036273
long                    0.054116
sqft_living15           0.246058
sqft_lot15              0.027030
FID_RasterT_Extract1   -0.010785
Id_1                   -0.010785
gridcode                     NaN
Name: price, dtype: float64

In [57]:
#Filter for highly correlated values
highly_corr_attributes = corr_price_highprice[abs(corr_price_highprice) > 0.5].index
highly_corr_attributes

Index(['price', 'bathrooms', 'sqft_living', 'grade', 'sqft_above'], dtype='object')